In [110]:
from rdkit import DataStructs, Chem
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from pandas import pandas
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit import ML
from rdkit.ML import Cluster
from rdkit.ML.Cluster import Butina


smiles = []
with open("Smiles.csv", "r") as ins:
    for line in ins:
        smiles.append(line.split('\n')[0])
print('# of SMILES:', len(smiles))
moles = []
for i in smiles:
    moles.append(Chem.MolFromSmiles(i))

radius=3
nBits=500000
Fingerprints = [AllChem.GetMorganFingerprintAsBitVect(x,3 , nBits=nBits) for x in moles]

# of SMILES: 2239


In [111]:
def ClusterFps(fps,cutoff):
    from rdkit import DataStructs
    from rdkit.ML.Cluster import Butina

    # first generate the distance matrix:
    dists = []
    nfps = len(fps)
    for i in range(1,nfps):
        sims = DataStructs.BulkTanimotoSimilarity(fps[i],fps[:i])
        dists.extend([1-x for x in sims])

    # now cluster the data:
    cs = Butina.ClusterData(dists,nfps,cutoff,isDistData=True)
    return cs
clusters = ClusterFps(Fingerprints, cutoff=0.45)


In [141]:
#show one of the clusters


#now display structures from one of the clusters
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
#print(len(Fingerprints))
#print(len(clusters))
# for i in range(len(clusters)):
#    print(len(clusters[i]))
#
newfingers = []
for i in range(len(clusters[0])):
   newfingers.append(Fingerprints[clusters[0][i]])
newsmiles = []
for i in range(len(clusters[0])):
   newsmiles.append(smiles[clusters[0][i]])

import pandas as pd
df = pd.DataFrame (newsmiles, columns = ['smiles'])
df.head()
df.to_csv('ClusteredData.csv')